In [24]:
#importin the necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier # this is the function that ensembles the model
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,confusion_matrix

In [25]:
#importing the dataset
dataset = pd.read_csv("D:\\Datascience\\SSN SACE\Project\\Kaggle\\Titanic - Machine learning from disaster\\train.csv")
dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


The features Pclass, sex age, sibsp, parch are considered for training since, model accuracy was high with previous runs of Logestic regression, KNN 
and Decision tree models

from EDA we know age has null values and these values are replaced by median value of age of grouped features of sex, pclass and title
from titles we can know whether passenger is adult or kid hence impute apporximate values

In [26]:
#extraction of titles from name
dataset['Title']=dataset['Name'].apply(lambda name:name.split(',')[1].split('.')[0].strip())
dataset['Title'].value_counts()

Mr              517
Miss            182
Mrs             125
Master           40
Dr                7
Rev               6
Mlle              2
Major             2
Col               2
the Countess      1
Capt              1
Ms                1
Sir               1
Lady              1
Mme               1
Don               1
Jonkheer          1
Name: Title, dtype: int64

In [27]:
#grouping of features
grouped_gender_pclass_title=dataset.groupby(['Sex','Pclass','Title'])
grouped_gender_pclass_title['Age'].count()

Sex     Pclass  Title       
female  1       Dr                1
                Lady              1
                Miss             45
                Mlle              2
                Mme               1
                Mrs              34
                the Countess      1
        2       Miss             32
                Mrs              41
                Ms                1
        3       Miss             69
                Mrs              33
male    1       Capt              1
                Col               2
                Don               1
                Dr                3
                Jonkheer          1
                Major             2
                Master            3
                Mr               87
                Sir               1
        2       Dr                2
                Master            9
                Mr               82
                Rev               6
        3       Master           24
                Mr              229

In [28]:
#imputing null values with median of grouped values
grouped_gender_pclass_title['Age'].median()
dataset['Age'] = grouped_gender_pclass_title['Age'].apply(lambda x: x.fillna(x.median()))

In [29]:
#while evacuating the ship kids, ladies and honorable passengers are given top priority hence considered in model
#titles are grouped based on honarable passengers, crews, kids and adults
standardized_title={'Mr':'Mr','Miss':'Miss','Mrs':'Mrs','Master':'Master','Dr':'Crew','Rev':'Honor','Mlle':'Miss','Major':'Crew','Col':'Crew',
                    'the Countess':'Honor','Capt':'Crew','Ms':'Miss','Sir':'Honor','Lady':'Honor','Mme':'Mrs','Don':'Honor','Jonkheer':'Honor'}
dataset['Title'] = dataset['Title'].map(standardized_title)

In [30]:
#ordinal encoding is considered for categorical features, since for this scenario ordinal encoding best fits than others
pclass=[1,2,3]
gender=['female','male']
title=['Honor','Master','Miss','Mrs','Mr','Crew']

In [31]:
#applying ordinal encoding
o=OrdinalEncoder(categories=[pclass,gender,title])
dataset[['Pclass','Sex','Title']]=o.fit_transform(dataset[['Pclass','Sex','Title']])

In [32]:
#checking the dataset
dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,2.0,"Braund, Mr. Owen Harris",1.0,22.0,1,0,A/5 21171,7.2500,NaN,S,4.0
1,2,1,0.0,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0.0,38.0,1,0,PC 17599,71.2833,C85,C,3.0
2,3,1,2.0,"Heikkinen, Miss. Laina",0.0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,2.0
3,4,1,0.0,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0.0,35.0,1,0,113803,53.1000,C123,S,3.0
4,5,0,2.0,"Allen, Mr. William Henry",1.0,35.0,0,0,373450,8.0500,NaN,S,4.0


In [34]:
#features considered for model
x = dataset[['Pclass','Sex','Age','SibSp','Parch','Title']]
y = dataset['Survived']

In [35]:
#spliting the data for training and testing
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=1000)

In [36]:
#fitting and training the dataset to ensemble model using LR, DCT and KNN classifier
model1 = LogisticRegression()
model2 = DecisionTreeClassifier(random_state=500)
model3 = KNeighborsClassifier(23)
model = VotingClassifier(estimators=[('lr', model1), ('dt', model2),('knn',model3)], voting='soft') # it will stitich all the models together, 
#voting = hard means max voting 
model.fit(x_train,y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('dt', DecisionTreeClassifier(random_state=500)),
                             ('knn', KNeighborsClassifier(n_neighbors=23))],
                 voting='soft')

In [37]:
#pedicting the test set
preds = model.predict(x_test)
preds

array([0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 1, 1], dtype=int64)

In [38]:
#checking confusion matrix and accuracy score
acc=accuracy_score(y_test, preds)
cm=confusion_matrix(y_test,preds)
print(acc)
print(cm)

0.8156424581005587
[[88 12]
 [21 58]]


Random forest classifier bas better accucracy than ensemble model